Hi, everyone! I'm a newcomer to this competition, and I'm here to learn as much as I can. In this notebook, I try to use LLM to generate main idea of questions at first, and use the idea as input features in the first time retrieval. In the second time retrieval, I divide the input features into two parts, and then searched for the top-25 separately. 

Finally, the search results are combined, and the results are sorted and output according to similarity.

But it's a pity that cv score decrease.

--------------------------

If you find this helpful, please consider voting for [takanashihumbert](https://www.kaggle.com/code/takanashihumbert/eedi-qwen-2-5-32b-awq-two-time-retrieval#LLM-Reasoning) as well:

* The main idea of this notebook is using retrieval two times.
  * The first time: Get the top-K1 relavent misconceptions to LLM as a reference(using ConstructName + SubjectName).
  * The second time: Get the top-K2(K2 < K1) relavent misconceptions(using ConstructName + SubjectName + Question + Answer + LLM's output).
  * Inference time: ~2 hours
  
  

Thanks to these great works:
- [Zero-shot w/ LLM feature (LB: 0.180)](https://www.kaggle.com/code/ubamba98/eedi-zero-shot-w-llm-feature-lb-0-180)
- [Infer BGE Synthetic Data](https://www.kaggle.com/code/minhnguyendichnhat/infer-bge-synthetic-data)
- [Fine-tuning bge Train](https://www.kaggle.com/code/sinchir0/fine-tuning-bge-train)

## cv and lb score



**Here is the table about running result.**

| Version | CV_score | LB_score |
|------|------------|------------|
| 1    | 0.3537187876767431 | 0.373 |


In [1]:
%%time
# 卸载名为torch的包
!pip uninstall -y torch
# 命令安装名为vllm的包
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
# 升级名为grpcio的包
!pip install -q -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# 升级名为ray的包
!pip install -q -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
# 安装名为sentence_transformers的包
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
CPU times: user 2.18 s, sys: 530 ms, total: 2.71 s
Wall time: 3min 34s


In [9]:
import os, math, numpy as np
import os
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc    # gc负责垃圾回收
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0,1" # 指定哪些 GPU 设备对当前 Python 进程可见
pd.set_option('display.max_rows', 300)""  # 控制输出行数

SyntaxError: invalid syntax (2228423496.py, line 9)

## Metric

评价函数

In [ ]:
%%writefile eedi_metrics.py
# 写一个评价函数的python文件
# 这实现的是官方的MAP@25评价
# Credit: https://www.kaggle.com/code/abdullahmeda/eedi-map-k-metric

import numpy as np
# Average Precision at k，简称AP@k
def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    # 没有真实值
    if not actual:
        return 0.0
    # 截断使得最多只有k个预测值
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    # 计算预测量中有多少个量 匹配 实际量
    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction，预测是正确的
        # second condition checks if prediction is not repeated，预测不是重复的
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
          

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Prepare dataframe，改IS_SUBMISSION为0来查看cv分数

数据预处理

In [ ]:
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN")) # 判断是初次运行还是多次运行
# 载入模型的路径，这里使用qwen2.5，阿里千问
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
# 分词器，有待了解
tokenizer = AutoTokenizer.from_pretrained(model_path)
# 取100个样本当训练集，空值都填了-1
df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).sample(100, random_state=42).reset_index(drop=True)
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")

# first retrieval
第一次检索

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

if not IS_SUBMISSION:
    # 如果是第一次运行
    df_ret = df_train.copy()
else:
    # 不是第一次运行就提交测试
    df_ret = df_test.copy()
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model = SentenceTransformer('/kaggle/input/eedi-finetuned-bge-public/Eedi-finetuned-bge')
# model = SentenceTransformer('/kaggle/input/fine-tuning-bge-train/trained_model')
df_ret.head()

In [ ]:
# 这个函数的目的是清洗文本数据，使其更适合后续的文本分析或机器学习任务。
# 通过删除特殊字符、URL、提及等，并将所有文本转换为小写，可以减少数据的噪声，使得特征提取更加准确。
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub("@\w+", '',x)      # Delete strings starting with @
    #x = re.sub("'\d+", '',x)      # Delete Numbers
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\times+", "\\\\times", x)  # Replace 'times' with '\times' with spaces around
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

In [ ]:
# 希望能提取问题的关键词
'''
比如：
Q——问题：
ConstructName：Simplify an algebraic fraction by factorising the numerator
SubjectName：BIDMAS
QuestionText："\[3 \times 2+4-5\] Where do the brackets need to go to make the answer equal \( 13 \) ?"
A——答案：
Understanding and applying the order of operations to achieve a specific result.
'''

firstPROMPT  = """
You are a Mathematics master. Your task is to succinctly summarize the core math knowledge from the information. 
Use your own words to convey the essence without directly referencing the problem's text, especially number in the text.

Infomation：
Here is a question about {ConstructName}({SubjectName}).
Question: {Question}

Example:
Infomation：
Here is a question about Simplify an algebraic fraction by factorising the numerator(BIDMAS).
Question: \[3 \times 2+4-5\] Where do the brackets need to go to make the answer equal \( 13 \) ?
Your answer: Understanding and applying the order of operations to achieve a specific result.
"""
def apply_keywordTemplate(row, tokenizer):
    # 有关于聊天模板的设置
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                firstPROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text
df_keyword = {}
for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
    df_keyword[f"{row.QuestionId}"] = apply_keywordTemplate(row, tokenizer)


df_keyword = pd.DataFrame([df_keyword]).T.reset_index()
df_keyword.columns = ["QuestionId", "text"]    # 存储 问题id、prompt
df_keyword.to_parquet("forKeyword.parquet", index=False)

In [ ]:
# 输出 df_keyword DataFrame 中第一行的 text 列的值
print(df_keyword.loc[0, 'text'])

In [ ]:
%%writefile run_vllmKeyword.py
# 新建大语言模型的提取关键词文件
import re
import vllm
import pandas as pd

df = pd.read_parquet("forKeyword.parquet")

model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

llm = vllm.LLM(
    model_path,
    quantization="awq",          # 动态量化，即在模型推理时动态地对权重进行量化
    tensor_parallel_size=2,      # 设置张量并行的大小，意味着模型将在两个设备上并行处理数据
    gpu_memory_utilization=0.90, # 模型将尝试使用最多90%的可用GPU内存
    trust_remote_code=True,      # 模型将执行来自远程源的代码
    dtype="half",                # 16位浮点数
    enforce_eager=True,          # 急切执行（Eager Execution）是一种立即执行操作的模式
    max_model_len=5120,          # 设置模型的最大生成长度，单位通常是令牌数
    disable_log_stats=True       # 禁用日志
)
# 获取模型的分词器（tokenizer）
tokenizer = llm.get_tokenizer()


responses = llm.generate(
    df["text"].values,  # 传入prompt
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.每个提示返回的输出序列数量
        top_p=0.8,  # Float that controls the cumulative probability of the top tokens to consider.顶部令牌的累积概率
        temperature=0.1,  # randomness of the sampling.控制采样的随机性
        seed=777, # Seed for reprodicibility.随机种子
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=1024,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm=True
)

responses = [x.outputs[0].text for x in responses]
# 新增一列
df["fullLLMText"] = responses

def extract_response(text):   # 没用上
    return ",".join(re.findall(r"<response>(.*?)</response>", text)).strip()
# 新增一列，代表llm给出的误差项
df["llmKeyword"] = responses
df.to_parquet("forKeyword.parquet", index=False)

In [ ]:
!python run_vllmKeyword.py

In [ ]:
llm_output = pd.read_parquet("forKeyword.parquet")

for idx, row in llm_output[0:5].iterrows():
    print(row.llmKeyword)
    print("==="*6)

In [ ]:
for idx, row in llm_output[0:5].iterrows():
    print(row.text)
    print("==="*6)

In [ ]:
def clean_keyword(row):
    flag = 0
    if "your answer:" in row['llmKeyword']:
        flag = 1
    text = row['llmKeyword'].strip()
    # 假设text 为 "your answer: mastering the subtraction of decimals with varying decimal places to achieve accurate results."
    if flag:
        # 删去text中的"your answer:"
        text = text.replace("your answer:", "").strip()
    return text

In [ ]:
llm_output['llmKeywordCleaned'] = llm_output.apply(clean_keyword, axis=1)
for idx, row in llm_output[0:5].iterrows():
    print(row.llmKeywordCleaned)
    print("==="*6)

In [ ]:
# 按列拼接，之后清理
df_ret['input_features'] = df_ret["ConstructName"] + ". " + df_ret["SubjectName"] + ". " + llm_output['llmKeywordCleaned']
df_ret['input_features'] = df_ret['input_features'].apply(lambda x: preprocess_text(x))


In [ ]:
# 对输入特征进行编码
embedding_query = model.encode(df_ret['input_features'], convert_to_tensor=True)
# 返回numpy数组，存储MisconceptionName的values
misconceptions = df_misconception_mapping.MisconceptionName.values
# 对误差项进行编码
embedding_Misconception = model.encode(misconceptions, convert_to_tensor=True)

# the first time retrieval for LLM prompt
# 简单理解，就是抛出输入特征，找到前k个比较符合输入特征方向的误差项解释
# 它计算 embedding_query 和 embedding_Misconception 中每个嵌入之间的相似度。
# 根据相似度分数，它对所有误区进行排序。
# 它返回相似度最高的前100个误区的ID。
Ret_topNids = util.semantic_search(embedding_query, embedding_Misconception, top_k=100)

In [ ]:
retrivals = []    # 用于存储每个问题的检索结果
dicts = {}        # 存储每个问题的检索误区的详细信息
for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
    top_ids = Ret_topNids[idx]
    retrival = ''                       # 对于每一行，从 Ret_topNids 列表中获取对应的最相关的误区ID列表
    dicts[str(row['QuestionId'])] = {}  # 用于构建当前问题的检索结果
    for i, ids in enumerate(top_ids):
        # serial number + misconceptions
        retrival += f'{i+1}. ' + misconceptions[ids['corpus_id']] + '\n'
        # save retrieved misconceptions for each QuestionId.
        dicts[str(row['QuestionId'])][str(i+1)] = misconceptions[ids['corpus_id']]
    retrivals.append(retrival)

# 把检索加到了后面
df_ret['Retrival'] = retrivals


In [ ]:
def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\times+", "\\\\times", x)  # Replace 'times' with '\times' 
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
No need to give the reasoning process and do not use "The misconception is" to start your answers.
There are some relative and possible misconceptions below to help you make the decision:

{Retrival}
"""
# just directly give your answers.
# 行（对应df_ret中每一行），向量化文本类，目标列（对应不正确的答案）
def apply_template(row, tokenizer, targetCol):
    # 有关于聊天模板的设置
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"Answer{targetCol}Text"],
                    CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"],
                    Retrival=row[f"Retrival"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

df = {}
if not IS_SUBMISSION:
    # 如果是第一次运行
    df_label = {}
    for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
        for option in ["A", "B", "C", "D"]:
            # 如果当前option不是正确答案且当前option有误解项
            if (row.CorrectAnswer!=option) & (row[f"Misconception{option}Id"]!=-1):
                # 生成模板并存储
                df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)
                df_label[f"{row.QuestionId}_{option}"] = [row[f"Misconception{option}Id"]]  # 提取真实值的
    # 将 df_label 字典转换为 DataFrame，并进行转置（.T），使得每个键值对成为一行，然后重置索引            
    df_label = pd.DataFrame([df_label]).T.reset_index()
    df_label.columns = ["QuestionId_Answer", "MisconceptionId"]  # 设置列名为 QuestionId_Answer 和 MisconceptionId
    df_label.to_parquet("label.parquet", index=False)
else:
    for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
        for option in ["A", "B", "C", "D"]:
            # 如果当前option不是正确答案
            if row.CorrectAnswer!=option:
                df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)

df = pd.DataFrame([df]).T.reset_index()
df.columns = ["QuestionId_Answer", "text"]    # 存储 问题id_错误答案、prompt
df.to_parquet("submission.parquet", index=False)

In [ ]:
# 输出 df DataFrame 中第一行的 text 列的值
print(df.loc[0, 'text'])

## LLM Reasoning
运用大语言模型推理

In [ ]:
%%writefile run_vllm.py

import re
import vllm
import pandas as pd

df = pd.read_parquet("submission.parquet")

model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

llm = vllm.LLM(
    model_path,
    quantization="awq",          # 动态量化，即在模型推理时动态地对权重进行量化
    tensor_parallel_size=2,      # 设置张量并行的大小，意味着模型将在两个设备上并行处理数据
    gpu_memory_utilization=0.90, # 模型将尝试使用最多90%的可用GPU内存
    trust_remote_code=True,      # 模型将执行来自远程源的代码
    dtype="half",                # 16位浮点数
    enforce_eager=True,          # 急切执行（Eager Execution）是一种立即执行操作的模式
    max_model_len=5120,          # 设置模型的最大生成长度，单位通常是令牌数
    disable_log_stats=True       # 禁用日志
)
# 获取模型的分词器（tokenizer）
tokenizer = llm.get_tokenizer()


responses = llm.generate(
    df["text"].values,  # 传入prompt
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.每个提示返回的输出序列数量
        top_p=0.8,  # Float that controls the cumulative probability of the top tokens to consider.顶部令牌的累积概率
        temperature=0.1,  # randomness of the sampling.控制采样的随机性
        seed=777, # Seed for reprodicibility.随机种子
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=1024,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm=True
)

responses = [x.outputs[0].text for x in responses]
# 新增一列
df["fullLLMText"] = responses

def extract_response(text):   # 没用上
    return ",".join(re.findall(r"<response>(.*?)</response>", text)).strip()
# 新增一列，代表llm给出的误差项
df["llmMisconception"] = responses
df.to_parquet("submission.parquet", index=False)

In [ ]:
!python run_vllm.py

In [ ]:
llm_output = pd.read_parquet("submission.parquet")

for idx, row in llm_output[0:5].iterrows():
    print(row.llmMisconception)
    print("==="*6)

```
1
==================
When dividing decimals with the same number of decimal places as each other, assumes the answer also has the same number of decimal places.
==================
11. Thinks index notation represents repeated addition rather than repeated multiplication
==================
Mixes up squaring and multiplying by 2 or doubling
==================
11. When rounding to decimal places and the only digits after the decimal point will be zeros, just gives the integer as the answer.
==================
```

In [ ]:
text = llm_output.loc[0, 'text']
PREFIX = "<|im_start|>user"
text = text.split(PREFIX)[1].split("You are a Mathematics teacher.")[0].strip('\n').split('Here is a question about')[-1].strip()
print(text)
# 提取问题

## Post-processing for LLM output
对生成的输出进行后续处理

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

df = pd.read_parquet("submission.parquet")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model = SentenceTransformer('/kaggle/input/eedi-finetuned-bge-public/Eedi-finetuned-bge')

In [ ]:
def number2sentence(row):
    """
    This is used for post-processing of LLM's output.
    Since we give top-N retrieval to the LLM with serial number,
    Sometimes the LLM will only output the serial number without any sentence.
    We use the 'dicts' generated at the beginning to map the serial number with corresponding misconceptions.
    LLM有时会只输出序列号而不输出句子，使用之前生成的dicts来映射序列号和对应的误区
    """
    text = row['llmMisconception'].strip()
    # potential is the most possible serial number in LLM output.
    potential = re.search(r'^\w+\.{0,1}', text).group()  # 选取text开头的语或词
    if '.' in potential:
        sentence = text.replace(potential, '').strip()   # 删去序号和小数点
    # if the LLM output is only a serial number, we map it with corresponding misconceptions saved in the dict.
    elif len(potential) == len(text):
        # 将这个问题的前N个可能的误解项提出来，以便之后映射
        qid_retrieval = dicts[row['QuestionId']]
        try:
            # qid_retrieval is the top-N misconceptions for an QuestionId,
            # qid_retrieval[potential] is the most possible misconception.
            sentence = qid_retrieval[potential]
        except:
            # If the mapping fails, we use the first one(the most possible one in the first retrieval).
            sentence = qid_retrieval['1']
    else:
        sentence = text
        
    return sentence


df['QuestionId'] = df['QuestionId_Answer'].apply(lambda x: x.split('_')[0])
# 进行误差项输出的处理
df['llmMisconception_clean'] = df.apply(number2sentence, axis=1)

In [ ]:
df.head(5)

# Second retrieval

In [ ]:
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub(r"@\w+", '',x)      # Delete strings starting with @
    #x = re.sub(r"\d+", '',x)      # Delete Numbers
    x = re.sub(r"http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\times+", "\\\\times", x)  # Replace 'times' with '\times' with spaces around
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PREFIX = "<|im_start|>user"
df['input_features'] = df["text"].apply(lambda x: x.split(PREFIX)[1].split("You are a Mathematics teacher.")[0].strip('\n').split('Here is a question about')[-1].strip())

df['input_features'] = df['input_features'].apply(lambda x: preprocess_text(x))


In [ ]:
df['input_features']


In [ ]:
df["llmMisconception_clean"]

In [ ]:
# Mixture
embedding_query = model.encode(df['input_features'], convert_to_tensor=True)
embedding_Misconception = model.encode(df_misconception_mapping.MisconceptionName.values, convert_to_tensor=True)
top25ids1 = util.semantic_search(embedding_query, embedding_Misconception, top_k=25)


In [ ]:
print(len(top25ids1))

In [ ]:

embedding_query = model.encode(df["llmMisconception_clean"], convert_to_tensor=True)
embedding_Misconception = model.encode(df_misconception_mapping.MisconceptionName.values, convert_to_tensor=True)
top25ids2 = util.semantic_search(embedding_query, embedding_Misconception, top_k=25)

In [ ]:
top25ids = []
for i in range(len(top25ids1)):
    top25id = top25ids1[i] + top25ids2[i]
    top25ids.append(top25id)

In [ ]:
len(top25ids)

In [ ]:
# 筛选分数
new_top25ids = []
for top25id in top25ids:
    top25idss = sorted(top25id, key=lambda x: x['score'], reverse=True)
    new_top25id = []
    for item in top25idss :
        if len(new_top25id) >= 25:
            break
        if item['score'] >= 0.6:
            new_top25id.append(item)
        else:
            break
    new_top25ids.append(new_top25id)

In [ ]:
len(new_top25ids)

In [ ]:
df["MisconceptionId"] = [" ".join([str(x["corpus_id"]) for x in top25id]) for top25id in new_top25ids]
# 输出提交文件
df[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)
df.head()

## Sanity

In [ ]:
# 计算误差
if not IS_SUBMISSION:
    import pandas as pd
    from eedi_metrics import mapk
    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])
    label = pd.read_parquet("label.parquet")["MisconceptionId"]
    print("Validation: ", mapk(label, predicted))